# TFX - Interactive Training Pipeline
1. Receive hyperparameters using hyperparam_gen custom python component
2. Extract data from BigQuery using BigQueryExampleGen
3. Validate the raw data using StatisticsGen and ExampleValidator
4. Process the data using Transform
5. Train a custom model using Trainer
6. Train an AutoML Tables model using automl_trainer custom python component
7. Evaluat the custom model using ModelEvaluator
8. Validate the custom model against the AutoML Tables model using a custom python component
7. Save the blessed to model registry location using using Pusher
8. Upload the model to AI Platform using aip_model_pusher custom python component

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import numpy as np
import tfx
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
from tensorflow_transform.tf_metadata import schema_utils
import logging

from model_src import data, features
from tfx_pipeline import components

logging.getLogger().setLevel(logging.INFO)

print("Tensorflow Version:", tf.__version__)

Tensorflow Version: 2.3.0


In [3]:
PROJECT = 'ksalama-cloudml'
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us'

DATASET_DISPLAYNAME = 'chicago_taxi_tips'
CUSTOM_MODEL_DISPLAYNAME = f'{DATASET_DISPLAYNAME}_classifier_custom'
AUTOML_MODEL_DISPLAYNAME = f'{DATASET_DISPLAYNAME}_classifier_automl'

LOCAL_WORKSPACE = '_workspace'
GCS_WORKSPACE = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/interactive"
RAW_SCHEMA_DIR = 'model_src/raw_schema'

ARTIFACT_STORE = os.path.join(GCS_WORKSPACE, 'tfx_artifacts')
MODEL_REGISTRY = os.path.join(GCS_WORKSPACE, 'model_registry')
MLMD_SQLLITE = os.path.join(LOCAL_WORKSPACE, 'mlmd.sqllite')
PIPELINE_NAME = f'{DATASET_DISPLAYNAME}_training_pipeline'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME)

!gcloud config set project $PROJECT

Updated property [core/project].


In [4]:
REMOVE_LOCAL_WORKSPACE = True

if tf.io.gfile.exists(LOCAL_WORKSPACE) and REMOVE_LOCAL_WORKSPACE:
    print("Removing previous local workspace...")
    tf.io.gfile.rmtree(LOCAL_WORKSPACE)

print("Creating new local workspace...")
tf.io.gfile.mkdir(LOCAL_WORKSPACE)

## Create Interactive Context

In [5]:
CLEAN_ARTIFACTS = True
if tf.io.gfile.exists(ARTIFACT_STORE) and CLEAN_ARTIFACTS:
    print("Removing previous artifacts...")
    tf.io.gfile.rmtree(ARTIFACT_STORE)

if not tf.io.gfile.exists(ARTIFACT_STORE):
    print("Creating local tfx artifact directory...")
    tf.io.gfile.mkdir(ARTIFACT_STORE)
    
print(f'Pipeline artifacts directory: {PIPELINE_ROOT}')
print(f'Local metadata SQLlit path: {MLMD_SQLLITE}')

Removing previous artifacts...
Creating local tfx artifact directory...
Pipeline artifacts directory: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline
Local metadata SQLlit path: _workspace/mlmd.sqllite


In [6]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = MLMD_SQLLITE
connection_config.sqlite.connection_mode = 3 # READWRITE_OPENCREATE
mlmd_store = mlmd.metadata_store.MetadataStore(connection_config)

context = InteractiveContext(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=connection_config
)

INFO:absl:MetadataStore with DB connection initialized


### 1. Hyperparameter Generation

In [7]:
hyperparams_gen = components.hyperparameters_gen(
    num_epochs=5,
    learning_rate=0.001,
    batch_size=512,
    hidden_units='64,64',
)

context.run(hyperparams_gen, enable_cache=False)

INFO:absl:Running driver for hyperparameters_gen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for hyperparameters_gen
INFO:root:Hyperparameters: {'num_epochs': 5, 'batch_size': 512, 'learning_rate': 0.001, 'hidden_units': [64, 64]}
INFO:root:Hyperparameters are written to: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/hyperparameters_gen/hyperparameters/63/hyperparameters.json
INFO:absl:Running publisher for hyperparameters_gen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: hyperparameters_gen
    execution_id: 63
    outputs:
        hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 78
        type_id: 5
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/hyperparameters_gen/hyperparameters/63"
        custom_properties {
          key: "name"
          value {
            string_value: "hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "hyperparameters_gen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "HyperParameters"
        )]
        ))

In [8]:
json.load(
    tf.io.gfile.GFile(
        os.path.join(
            hyperparams_gen.outputs.hyperparameters.get()[0].uri, 'hyperparameters.json')
    )
)

{'num_epochs': 5,
 'batch_size': 512,
 'learning_rate': 0.001,
 'hidden_units': [64, 64]}

## 2. Data Extraction

In [9]:
from utils import datasource_utils
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen
from tfx.proto import example_gen_pb2, transform_pb2

### Extract train and eval splits

In [10]:
sql_query = datasource_utils.get_source_query(
    PROJECT, REGION, DATASET_DISPLAYNAME, data_split='UNASSIGNED', limit=10000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
        ]
    )
)

train_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location=gs://{BUCKET}/bq_tmp"
]

context.run(
    train_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

INFO:absl:Running driver for BigQueryExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for BigQueryExampleGen
INFO:absl:Attempting to infer TFX Python dependency for beam
INFO:absl:Copying all content from install dir /opt/conda/lib/python3.7/site-packages/tfx to temp dir /tmp/tmpqdcvsfgb/build/tfx
INFO:absl:Generating a temp setup file at /tmp/tmpqdcvsfgb/build/tfx/setup.py
INFO:absl:Creating temporary sdist package, logs available at /tmp/tmpqdcvsfgb/build/tfx/setup.log
INFO:absl:Added --extra_package=/tmp/tmpqdcvsfgb/build/tfx/dist/tfx_ephemeral-0.26.0.tar.gz to beam args
INFO:absl:Generating examples.


/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1881: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function annotate_downstream_side_inputs at 0x7f2d0799c680> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function fix_side_input_pcoll_coders at 0x7f2d0799c7a0> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function lift_combiners at 0x7f2d0799ca70> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function expand_sdf at 0x7f2d0799cc20> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function expand_gbk at 0x7f2d0799

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 64
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 79
        type_id: 7
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/BigQueryExampleGen/examples/64"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "BigQueryExampleGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

### Extract test split

In [11]:
sql_query = datasource_utils.get_source_query(
    PROJECT, REGION, DATASET_DISPLAYNAME, data_split='TEST', limit=1000)

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=1),
        ]
    )
)

test_example_gen = BigQueryExampleGen(query=sql_query, output_config=output_config)

beam_pipeline_args=[
    f"--project={PROJECT}",
    f"--temp_location=gs://{BUCKET}/bq_tmp"
]

context.run(
    test_example_gen,
    beam_pipeline_args=beam_pipeline_args,
    enable_cache=False
)

INFO:absl:Running driver for BigQueryExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for BigQueryExampleGen
INFO:absl:Attempting to infer TFX Python dependency for beam
INFO:absl:Copying all content from install dir /opt/conda/lib/python3.7/site-packages/tfx to temp dir /tmp/tmpoutryavz/build/tfx
INFO:absl:Generating a temp setup file at /tmp/tmpoutryavz/build/tfx/setup.py
INFO:absl:Creating temporary sdist package, logs available at /tmp/tmpoutryavz/build/tfx/setup.log
INFO:absl:Added --extra_package=/tmp/tmpoutryavz/build/tfx/dist/tfx_ephemeral-0.26.0.tar.gz to beam args
INFO:absl:Generating examples.
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function annotate_downstream_side_inputs at 0x7f2d0799c680> ====================
INFO:apache_beam.runners.portability.fn_api_runner.translations:==================== <function fix_side_input_pcoll_coders at 0x7f2d0799c7a0> ====================
INFO:apache_be

ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 65
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 80
        type_id: 7
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/BigQueryExampleGen/examples/65"
        properties {
          key: "split_names"
          value {
            string_value: "[\"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "BigQueryExampleGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [12]:
train_uri = os.path.join(train_example_gen.outputs.examples.get()[0].uri, "train/*")
print(train_uri)

source_raw_schema = tfdv.load_schema_text(os.path.join(RAW_SCHEMA_DIR, 'schema.pbtxt'))
raw_feature_spec = schema_utils.schema_as_feature_spec(source_raw_schema).feature_spec

def _parse_tf_example(tfrecord):
    return tf.io.parse_single_example(tfrecord, raw_feature_spec)

tfrecord_filenames = tf.data.Dataset.list_files(train_uri)
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
dataset = dataset.map(_parse_tf_example)

for raw_features in dataset.shuffle(1000).batch(3).take(1):
    for key in raw_features:
        print(f"{key}: {np.squeeze(raw_features[key], -1)}")
    print("")

gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/BigQueryExampleGen/examples/64/train/*
dropoff_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.6 42)']
euclidean: [   0.     7630.887  3377.9248]
loc_cross: [b'POINT(-87.7 42)POINT(-87.7 42)' b'POINT(-87.7 42)POINT(-87.7 42)'
 b'POINT(-87.7 42)POINT(-87.6 42)']
payment_type: [b'Cash' b'Prcard' b'Cash']
pickup_grid: [b'POINT(-87.7 42)' b'POINT(-87.7 42)' b'POINT(-87.7 42)']
tip_bin: [0 0 0]
trip_day: [ 5 12 13]
trip_day_of_week: [4 1 5]
trip_hour: [19 11 13]
trip_miles: [ 0.05 10.19  2.11]
trip_month: [2 1 2]
trip_seconds: [  47 5057  381]
trip_start_timestamp: [b'2020-02-05 19:00:00+00' b'2020-01-12 11:45:00+00'
 b'2020-02-13 13:45:00+00']



## 3. Data Validation

### Import raw schema

In [13]:
schema_importer = tfx.components.common_nodes.importer_node.ImporterNode(
    instance_name='Schema_Importer',
    source_uri=RAW_SCHEMA_DIR,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False
)

context.run(schema_importer)

INFO:absl:Running driver for ImporterNode.Schema_Importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: model_src/raw_schema, properties: {}, custom_properties: {}
INFO:absl:Reusing existing artifact
INFO:absl:Running executor for ImporterNode.Schema_Importer
INFO:absl:Running publisher for ImporterNode.Schema_Importer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 66
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 9
        uri: "model_src/raw_schema"
        create_time_since_epoch: 1618237944356
        last_update_time_since_epoch: 1618238037036
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

### Generate statistics

In [14]:
statistics_gen = tfx.components.StatisticsGen(
    instance_name='Statistics_Generation',
    examples=train_example_gen.outputs.examples)
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen.Statistics_Generation
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen.Statistics_Generation
INFO:absl:Generating statistics for split train.
INFO:apache_beam.io.gcp.gcsio:Starting the size estimation of the input
INFO:apache_beam.io.gcp.gcsio:Finished listing 1 files in 0.04933309555053711 seconds.
INFO:absl:Statistics for split train written to gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/StatisticsGen.Statistics_Generation/statistics/67/train.
INFO:absl:Generating statistics for split eval.
INFO:apache_beam.io.gcp.gcsio:Starting the size estimation of the input
INFO:apache_beam.io.gcp.gcsio:Finished listing 1 files in 0.040190696716308594 seconds.
INFO:absl:Statistics for split eval written to gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interact

ExecutionResult(
    component_id: StatisticsGen.Statistics_Generation
    execution_id: 67
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 81
        type_id: 11
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/StatisticsGen.Statistics_Generation/statistics/67"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen.Statistics_Generation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [15]:
!rm -r {RAW_SCHEMA_DIR}/.ipynb_checkpoints/

rm: cannot remove 'model_src/raw_schema/.ipynb_checkpoints/': No such file or directory


### Validate statistics against schema

In [16]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs.statistics,
    schema=schema_importer.outputs.result,
    instance_name="Data_Validation"
)

context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator.Data_Validation
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator.Data_Validation
INFO:absl:Validating schema against the computed statistics for split train.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Validation complete for split train. Anomalies written to gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/ExampleValidator.Data_Validation/anomalies/68/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/ExampleValidator.Data_Validation/anomalies/68/eval.
INFO:absl:Running publisher for ExampleValidator.Data_Validation
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 68
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 82
        type_id: 13
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/ExampleValidator.Data_Validation/anomalies/68"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator.Data_Validation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [17]:
context.show(example_validator.outputs.anomalies)

## 4. Data Transformation

In [18]:
_transform_module_file = 'model_src/preprocessing.py'

transform = tfx.components.Transform(
    examples=train_example_gen.outputs.examples,
    schema=schema_importer.outputs.result,
    module_file=_transform_module_file,
    splits_config=transform_pb2.SplitsConfig(
        analyze=['train'], transform=['train', 'eval']),
    instance_name="Data_Transformation"
)

context.run(transform, enable_cache=False)

INFO:absl:Running driver for Transform.Data_Transformation
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform.Data_Transformation


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
INFO:absl:Loading model_src/preprocessing.py because it has not been loaded before.
INFO:absl:model_src/preprocessing.py is already loaded.
INFO:absl:Feature trip_start_timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature payment_type has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature pickup_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature dropoff_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature loc_cross has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_month has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day_of_week has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_hour has a shape dim {
  

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.
INFO:absl:Feature trip_start_timestamp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature payment_type has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature pickup_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature dropoff_grid has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature loc_cross has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_month has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_day_of_week has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_hour has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_seconds has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature trip_miles has a shape dim {
  size: 1
}
. Setting to DenseTensor.


...

value: "\n\014\n\nConst_17:0\022\tloc_cross"



value: "\n\014\n\nConst_17:0\022\tloc_cross"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:apache_beam.runners.portability.fn_api_runner.fn_runner:Running (((ref_AppliedPTransform_Transform[TransformIndex1]/PrepareToClearSharedKeepAlives/Impulse_1090)+(ref_AppliedPTransform_Transform[TransformIndex1]/PrepareToClearSharedKeepAlives/FlatMap(<lambda at core.py:2957>)_1091))+(ref_AppliedPTransform_Transform[TransformIndex1]/PrepareToClearSharedKeepAlives/Map(decode)_1093))+(ref_AppliedPTransform_Transform[TransformIndex1]/WaitAndClearSharedKeepAlives_1094)
INFO:apache_beam.runners.portability.fn_api_runner.fn_runner:Running ((WriteCache/Write[AnalysisIndex0][CacheKeyIndex0]/Write/WriteImpl/GroupByKey/Read)+(ref_AppliedPTransform_WriteCache/Write[AnalysisIndex0][CacheKeyIndex0]/Write/WriteImpl/Extract_873))+(ref_PCollection_PCollection_496/Write)
INFO:apache_beam.runners.portability.fn_api_runner.fn_runner:Running ((ref_PCollection_PCollection_490/Read)+(ref_AppliedPTransform_WriteCache/

ExecutionResult(
    component_id: Transform.Data_Transformation
    execution_id: 69
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 83
        type_id: 15
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Transform.Data_Transformation/transform_graph/69"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform.Data_Transformation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 84
        type_id: 7
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Transform.Data_Transformation/transformed_examples/69"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform.Data_Transformation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 85
        type_id: 16
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Transform.Data_Transformation/updated_analyzer_cache/69"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform.Data_Transformation"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "TransformCache"
        )]
        ))

In [19]:
train_uri = os.path.join(transform.outputs.transformed_examples.get()[0].uri, "train/*")
transform_graph_uri = transform.outputs.transform_graph.get()[0].uri

tft_output = tft.TFTransformOutput(transform_graph_uri)
transform_feature_spec = tft_output.transformed_feature_spec()

for input_features, target in data.get_dataset(
    train_uri, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} ({input_features[key].dtype}): {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

dropoff_grid_xf (<dtype: 'int64'>): [0, 0, 0]
euclidean_xf (<dtype: 'float32'>): [1.8903874158859253, -0.36246100068092346, -0.7740612030029297]
loc_cross_xf (<dtype: 'int64'>): [1, 0, 0]
payment_type_xf (<dtype: 'int64'>): [2, 0, 0]
pickup_grid_xf (<dtype: 'int64'>): [1, 0, 0]
trip_day_of_week_xf (<dtype: 'int64'>): [2, 4, 3]
trip_day_xf (<dtype: 'int64'>): [5, 24, 26]
trip_hour_xf (<dtype: 'int64'>): [1, 1, 22]
trip_miles_xf (<dtype: 'float32'>): [0.06116146594285965, -0.5794121623039246, -0.24639876186847687]
trip_month_xf (<dtype: 'int64'>): [0, 1, 0]
trip_seconds_xf (<dtype: 'float32'>): [0.28780215978622437, -0.34918004274368286, -0.26991114020347595]
target: [0, 0, 0]


## 5. Custom Model Training

In [20]:
from tfx.components.base import executor_spec
from tfx.components.trainer import executor as trainer_executor

In [21]:
_train_module_file = 'model_src/runner.py'

trainer = tfx.components.Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=_train_module_file,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    train_args=tfx.proto.trainer_pb2.TrainArgs(num_steps=0),
    eval_args=tfx.proto.trainer_pb2.EvalArgs(num_steps=None),
    hyperparameters=hyperparams_gen.outputs.hyperparameters,
    instance_name='Model_Trainer'
)

context.run(trainer, enable_cache=False)

INFO:absl:Running driver for Trainer.Model_Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer.Model_Trainer
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:Loading model_src/runner.py because it has not been loaded before.
INFO:absl:Training model.
INFO:root:Runner started...
INFO:root:fn_args: FnArgs(working_dir=None, train_files=['gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Transform.Data_Transformation/transformed_examples/69/train/*'], eval_files=['gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Transform.Data_Transformation/transformed_examples/69/eval/*'], train_steps=None, eval_steps=None, schema_path='model_src/raw_schema/schema.pbtxt', schema_file='model_src/raw_schema/schema.pbtxt', transform_gr

Epoch 1/5
15/15 [==============================] 0.6065 - accuracy: 0.79 - 0s 4ms/step - loss: 0.5142 - accuracy: 0.8048 - 0s 29ms/step - loss: 0.5082 - accuracy: 0.8049 - val_loss: 0.4082 - val_accuracy: 0.8125
Epoch 2/5
15/15 [==============================] - ETA: 0s - loss: 0.4017 - accuracy: 0.80 - 0s 13ms/step - loss: 0.3537 - accuracy: 0.8129 - val_loss: 0.3049 - val_accuracy: 0.8359
Epoch 3/5
15/15 [==============================] - ETA: 0s - loss: 0.2948 - accuracy: 0.84 - 0s 13ms/step - loss: 0.2697 - accuracy: 0.8470 - val_loss: 0.2350 - val_accuracy: 0.8646
Epoch 4/5
15/15 [==============================] - ETA: 0s - loss: 0.2235 - accuracy: 0.86 - ETA: 0s - loss: 0.2144 - accuracy: 0.87 - 0s 13ms/step - loss: 0.2144 - accuracy: 0.8777 - val_loss: 0.2054 - val_accuracy: 0.8952
Epoch 5/5
15/15 [==============================] - ETA: 0s - loss: 0.1760 - accuracy: 0.91 - ETA: 0s - loss: 0.1869 - accuracy: 0.91 - 0s 13ms/step - loss: 0.1869 - accuracy: 0.9116 - val_loss: 0.1999

INFO:root:Model training completed.
INFO:root:Runner executing exporter...


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Trainer.Model_Trainer/model/70/serving_model_dir/assets


INFO:tensorflow:Assets written to: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Trainer.Model_Trainer/model/70/serving_model_dir/assets
INFO:root:Runner completed.


ExecutionResult(
    component_id: Trainer.Model_Trainer
    execution_id: 70
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 86
        type_id: 18
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Trainer.Model_Trainer/model/70"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer.Model_Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 87
        type_id: 19
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Trainer.Model_Trainer/model_run/70"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer.Model_Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 19
        name: "ModelRun"
        )]
        ))

## 6. AutoML Model Training

In [22]:
exclude_columns = ','.join(['trip_start_timestamp'])

automl_trainer = components.automl_trainer(
    project=PROJECT,
    region=REGION,
    dataset_display_name=DATASET_DISPLAYNAME,
    model_display_name=AUTOML_MODEL_DISPLAYNAME,
    target_column=features.TARGET_FEATURE_NAME,
    data_split_column='data_split',
    exclude_cloumns=exclude_columns,
    schema=schema_importer.outputs.result,
)

context.run(automl_trainer, enable_cache=False)

INFO:root:Retrieving chicago_taxi_tips dataset id...
INFO:root:chicago_taxi_tips dataset uri: projects/900786220115/locations/us-central1/datasets/5309093049786171392
INFO:root:chicago_taxi_tips dataset id: 5309093049786171392
INFO:root:Loading schema from: model_src/raw_schema/schema.pbtxt
INFO:root:AutoML Table training job specs: {'display_name': 'train_chicago_taxi_tips_classifier_automl_20210413115835', 'training_task_definition': 'gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_tabular_1.0.0.yaml', 'training_task_inputs': struct_value {
  fields {
    key: "disableEarlyStopping"
    value {
      bool_value: false
    }
  }
  fields {
    key: "optimizationObjective"
    value {
      string_value: "minimize-log-loss"
    }
  }
  fields {
    key: "predictionType"
    value {
      string_value: "classification"
    }
  }
  fields {
    key: "targetColumn"
    value {
      string_value: "tip_bin"
    }
  }
  fields {
    key: "trainBudgetMilliNodeHours"
    val

ExecutionResult(
    component_id: automl_trainer
    execution_id: 71
    outputs:
        uploaded_model: Channel(
            type_name: UploadedModel
            artifacts: [Artifact(artifact: id: 88
        type_id: 29
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/automl_trainer/uploaded_model/71"
        custom_properties {
          key: "model_uri"
          value {
            string_value: "projects/900786220115/locations/us-central1/models/7313230068338130944"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "uploaded_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "automl_trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "UploadedModel"
        )]
        ))

In [23]:
automl_trainer.outputs.uploaded_model.get()[0].get_string_custom_property('model_uri')

'projects/900786220115/locations/us-central1/models/7313230068338130944'

## 7. Custom Model Evaluation

In [24]:
from tfx.components import Evaluator

In [25]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name='serving_tf_example',
            label_key=features.TARGET_FEATURE_NAME,
            prediction_key='probabilities')
    ],
    slicing_specs=[
        tfma.SlicingSpec(),
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[   
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(
                    class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value': 0.8}))),
        ])
    ])


evaluator = Evaluator(
    examples=test_example_gen.outputs.examples,
    example_splits=['test'],
    model=trainer.outputs.model,
    eval_config=eval_config,
    schema=schema_importer.outputs.result
)

context.run(evaluator, enable_cache=False)

        please use "has_baseline" instead.
INFO:apache_beam.io.gcp.gcsio:Starting the size estimation of the input
INFO:apache_beam.io.gcp.gcsio:Finished listing 1 files in 0.039464473724365234 seconds.
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.Sequence[typing.Dict[str, typing.Any]]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.Sequence[typing.Dict[str, typing.Any]]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.Sequence[typing.Dict[str, typing.Any]]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.Sequence[typing.Dict[str, typing.Any]]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.Sequence[typing.Dict[str, typing.Any]]
INFO:apache_beam.typehints.native_type_compatibility:Using Any for unsupported type: typing.Sequence[typing.Dict[str, typing.Any]]
INFO:apache

ExecutionResult(
    component_id: Evaluator
    execution_id: 72
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 89
        type_id: 21
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Evaluator/evaluation/72"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelEvaluation"
        )]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 90
        type_id: 22
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Evaluator/blessing/72"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Trainer.Model_Trainer/model/70"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 86
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "ModelBlessing"
        )]
        ))

In [26]:
evaluation_results = evaluator.outputs.evaluation.get()[0].uri
print("validation_ok:", tfma.load_validation_result(evaluation_results).validation_ok)

for entry in list(tfma.load_metrics(evaluation_results))[0].metric_keys_and_values:
    print(entry.key.name, ":", round(entry.value.double_value.value, 3))

validation_ok: True
accuracy : 0.947
binary_accuracy : 0.947
example_count : 1000.0


## 8. Models Validation

### Get AutoML evaluation results

In [27]:
automl_metric_gen = components.automl_metrics_gen(
    project=PROJECT,
    region=REGION,
    uploaded_model=automl_trainer.outputs.uploaded_model
)

context.run(automl_metric_gen, enable_cache=False)

INFO:root:Retrieving metrics for model: projects/900786220115/locations/us-central1/models/7313230068338130944
INFO:root:Evaluation results: {'auPrc': 0.95514035, 'auRoc': 0.9516639, 'accuracy': 0.884572308553829, 'f1Score': 0.8845723}
INFO:root:Evaluation results are written to: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/automl_metrics_gen/evaluation/73/evaluation_results.json


ExecutionResult(
    component_id: automl_metrics_gen
    execution_id: 73
    outputs:
        evaluation: Channel(
            type_name: UploadedModelEvaluation
            artifacts: [Artifact(artifact: id: 91
        type_id: 31
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/automl_metrics_gen/evaluation/73"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "automl_metrics_gen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "UploadedModelEvaluation"
        )]
        ))

### Compare the evaluation results of the custom model and the AutoML model

In [30]:
validator = components.custom_model_validator(
    model_evaluation=evaluator.outputs.evaluation,
    uploaded_model_evaluation=automl_metric_gen.outputs.evaluation,
)

context.run(validator, enable_cache=False)

INFO:root:Loading model evaluation from: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Evaluator/evaluation/72
INFO:root:Model evaluation metrics: [key {
  name: "accuracy"
}
value {
  double_value {
    value: 0.9470000267028809
  }
}
, key {
  name: "binary_accuracy"
}
value {
  double_value {
    value: 0.9470000267028809
  }
}
, key {
  name: "example_count"
}
value {
  double_value {
    value: 1000.0
  }
}
]
INFO:root:Loading uploaded model evaluation from: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/automl_metrics_gen/evaluation/73
INFO:root:Blessing result True written to gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/custom_model_validator/blessing/75


ExecutionResult(
    component_id: custom_model_validator
    execution_id: 75
    outputs:
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 93
        type_id: 22
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/custom_model_validator/blessing/75"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "custom_model_validator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "ModelBlessing"
        )]
        ))

## 9. Model Pushing

In [31]:
exported_model_location = os.path.join(MODEL_REGISTRY, f'{DATASET_DISPLAYNAME}_classifier')

push_destination=tfx.proto.pusher_pb2.PushDestination(
    filesystem=tfx.proto.pusher_pb2.PushDestination.Filesystem(
        base_directory=exported_model_location,
    )
)

pusher = tfx.components.Pusher(
    model=trainer.outputs.model,
    #model_blessing=evaluator.outputs.blessing,
    model_blessing=validator.outputs.blessing,
    push_destination=push_destination
)

context.run(pusher, enable_cache=False)

ExecutionResult(
    component_id: Pusher
    execution_id: 76
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 94
        type_id: 24
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/Pusher/pushed_model/76"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/model_registry/chicago_taxi_tips_classifier/1618315787"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1618315787"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "PushedModel"
        )]
        ))

## 10. Model Upload to AI Platform

In [35]:
serving_runtime ='tf2-cpu.2-3'
serving_image_uri = f"gcr.io/cloud-aiplatform/prediction/{serving_runtime}:latest"

aip_model_uploader = components.aip_model_uploader(
    project=PROJECT,
    region=REGION,
    model_display_name=CUSTOM_MODEL_DISPLAYNAME,
    pushed_model_location=exported_model_location,
    serving_image_uri=serving_image_uri,
)

context.run(aip_model_uploader, enable_cache=False)

INFO:root:Model registry dir: gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/model_registry/chicago_taxi_tips_classifier/1618315787/
INFO:root:Model uploaded to AI Platform: projects/900786220115/locations/us-central1/models/4286811118745157632


ExecutionResult(
    component_id: aip_model_uploader
    execution_id: 79
    outputs:
        uploaded_model: Channel(
            type_name: UploadedModel
            artifacts: [Artifact(artifact: id: 97
        type_id: 29
        uri: "gs://ksalama-cloudml-us/ucaip_demo/chicago_taxi/tfx_artifacts_interactive/chicago_taxi_tips_training_pipeline/aip_model_uploader/uploaded_model/79"
        custom_properties {
          key: "model_uri"
          value {
            string_value: "projects/900786220115/locations/us-central1/models/4286811118745157632"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "uploaded_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "aip_model_uploader"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 29
        name: "UploadedModel"
        )]
        ))

In [ ]:
aip_model_uploader.outputs.uploaded_model.get()[0].get_string_custom_property('model_uri')